In [16]:
import pandas as pd
import great_expectations as gx
from great_expectations.validator.validator import Validator
from great_expectations.execution_engine.pandas_execution_engine import PandasExecutionEngine
from great_expectations.render.renderer import ValidationResultsPageRenderer
from great_expectations.render.view import DefaultJinjaPageView
import os
import webbrowser

## 1. Carregando o Dataset
Carregando o dataset a partir do arquivo raw no github.

In [ ]:

# url = 'https://raw.githubusercontent.com/thainazanfolin/great_expectations_tests/refs/heads/main/Sleep_Efficiency_table.csv'

df = pd.read_csv(r'C:\Users\thain\Desktop\great_x\Sleep_Efficiency_table.csv', delimiter=';')

# visão inicial do df
df.head()



## 2. Análise exploratória dos dados
Iniciando a análise exploratória para entender possíveis testes úteis no momento do ETL.  <br><br>
Em outras palavras, fazendo a análise exploratória dos dados, conseguimos **entender o que nós esperamos deles, ou seja, quais são nossas expectativas.**

In [ ]:
# número de linhas e colunas
print(f"\nLinhas e colunas: {df.shape[0]} linhas e {df.shape[1]} colunas")

# nomes das colunas
print("\nColunas:")
print(df.columns.tolist())

# tipos de dados e contagem (não nulos)
print("\nInfos:")
print(df.info())

# valores nulos por coluna
print("\nValores nulos por coluna:")
print(df.isnull().sum())

# % de valores nulos
print("\n% de valores nulos por coluna:")
print((df.isnull().mean() * 100).round(2))

# Colunas numéricas
print("\nEstatísticas descritivas (colunas numéricas):")
print(df.describe())

# Colunas categóricas
print("\nEstatísticas descritivas (colunas categóricas):")
print(df.describe(include=['object']))

# Detecta possíveis colunas com valores duplicados
print("\nRegistros duplicados (linhas inteiras):", df.duplicated().sum())

# Verifica se há colunas que parecem ser identificadores únicos
print("\nColunas com valores únicos em todas as linhas (possíveis IDs):")
for col in df.columns:
    if df[col].is_unique:
        print(f"- {col}")


# **Resultado da análise exploratória**
* 452 linhas e 15 colunas.

* Colunas: inclui variáveis sobre dados pessoais (ID, idade, gênero), hábitos de sono (hora de dormir, hora de acordar, duração do sono, eficiência do sono, percentagens de sono REM, profundo e leve, despertares), consumo de substâncias (cafeína, álcool, tabagismo) e exercícios físicos.

Ausência de Dados:

* Awakenings: 4.42% ausentes.

* Caffeine consumption: 5.53% ausentes.

* Alcohol consumption: 3.10% ausentes.

* Exercise frequency: 1.33% ausentes.


## **Usando o Great Expectations para ETL**

### O que vamos fazer com o great_expectations:

A partir dos resultados da análise exploratória, vemos que temos valores nulos em alguns campos e, além disso, conseguimos saber o que esperar (ou seja, nossas expectativas) frente aos dados que vamos receber (tipo de dado, range de valores etc). <br><br>
É isto que vamos fazer agora!

# Configurando o Great Expectations

In [18]:

df = pd.read_csv("Sleep_Efficiency_table.csv", delimiter=";")  # ajuste o caminho e delimitador se necessário

# contexto 
context = gx.get_context()

#  fonte de dados
datasource = context.sources.add_pandas(name="pandas_ds_11")

#  DataFrame como um asset
data_asset = datasource.add_dataframe_asset(name="sleep_data")

# batch a partir do DataFrame
batch_request = data_asset.build_batch_request(dataframe=df)

#  validador com base no batch
validator = context.get_validator(batch_request=batch_request)

## Começando as validações

In [ ]:
# 1. valores ausentes
validator.expect_column_values_to_not_be_null("Sleep efficiency")
validator.expect_column_values_to_not_be_null("Age")
validator.expect_column_values_to_not_be_null("Sleep duration")
validator.expect_column_values_to_not_be_null("Gender")
validator.expect_column_values_to_not_be_null("Bedtime")
validator.expect_column_values_to_not_be_null("Wakeup time")

# 2. dentro de intervalos válidos
validator.expect_column_values_to_be_between("Age", min_value=18, max_value=100)
validator.expect_column_values_to_be_between("Sleep efficiency", min_value=0.0, max_value=1.0)
validator.expect_column_values_to_be_between("Sleep duration", min_value=3, max_value=12)
validator.expect_column_values_to_be_between("Awakenings", min_value=0, max_value=10)
# validator.expect_column_values_to_be_greater_than_or_equal_to("Caffeine consumption", 0)
# validator.expect_column_values_to_be_greater_than_or_equal_to("Alcohol consumption", 0)
# validator.expect_column_values_to_be_greater_than_or_equal_to("Exercise frequency", 0)

# 3. verificacao de tipos
validator.expect_column_values_to_be_of_type("Age", "int64")
validator.expect_column_values_to_be_of_type("Sleep efficiency", "float64")

# 4. conjuntos de valores esperados
validator.expect_column_values_to_be_in_set("Gender", ["Male", "Female"])
validator.expect_column_values_to_be_in_set("Smoking status", ["Yes", "No"])

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 452,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## Executando e vendo as validações

In [ ]:
# executa validações
results = validator.validate()

# verificando os resultados
print("Resultados da validação:", results)

Calculating Metrics:   0%|          | 0/43 [00:00<?, ?it/s]

Resultados da validação: {
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "column": "Sleep efficiency",
          "batch_id": "pandas_ds_10-sleep_data"
        },
        "meta": {}
      },
      "result": {
        "element_count": 452,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_between",
        "kwargs": {
          "min_value": 0.0,
          "max_value": 1.0,
          "column": "Sleep efficiency",
          "batch_id": "pandas_ds_10-sleep_data"
        },
        "meta": {}
      },
      "resu

## Gerando o relatório

In [19]:
# Gera o relatório
context.build_data_docs()
# context.open_data_docs()

# 2. Gerar a renderização HTML do resultado de validação
document_model = ValidationResultsPageRenderer().render(results)  # Usando `results` corretamente
html = DefaultJinjaPageView().render(document_model)  # Renderizando o HTML

# 3. Criar o diretório de saída (se não existir) e salvar o HTML
output_dir = "ge_output"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "sleep_report.html")  # Usando a variável `output_path` corretamente

# # Salvar o HTML gerado no arquivo
# with open(output_path, "w") as f:
#     f.write(html)

# # Caminho do arquivo HTML gerado
# if os.path.exists(output_path):
#     webbrowser.open(f"file://{os.path.abspath(output_path)}")  # Abrindo o arquivo no navegador
# else:
#     print("Arquivo não encontrado.")
